In [1]:
library(DBI)
library(RSQLite)

soccer <- dbConnect(drv=RSQLite::SQLite(), dbname="../input/soccer/database.sqlite")

##### Assem's code
library(tidyverse)
library(reshape2)
library(data.table)
library(dplyr)
library(knitr)

## list all tables
tables <- dbListTables(soccer)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]
lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- 
    dbGetQuery(conn=soccer, 
               statement=paste("SELECT * FROM '", 
                               tables[[i]], "'", sep=""))
}

# create dataframes
country <-  data.frame(lDataFrames[1])
league  <-  data.frame(lDataFrames[2])
match   <-  data.frame(lDataFrames[3])
player  <-  data.frame(lDataFrames[4])
team    <-  data.frame(lDataFrames[6])

# select relevant columns

match <- match %>% select(match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal, away_team_goal)
team <- team %>% select(team_api_id,team_long_name)


match$home_score <- (match$home_team_goal - match$away_team_goal)
match$away_score <- - match$home_score
match$home_win_rate <- as.integer(as.logical(match$home_score>0))
match$away_win_rate <- as.integer(as.logical(match$away_score>0))


match_home <- match %>% select(match_api_id,date,home_team_api_id,home_score,home_win_rate) %>% 
  rename(
    team_api_id = home_team_api_id,
    score = home_score,
    win_rate = home_win_rate
    )
match_away <- match %>% select(match_api_id,date,away_team_api_id,away_score, away_win_rate) %>% 
  rename(
    team_api_id = away_team_api_id,
    score = away_score,
    win_rate = away_win_rate
    )

total <- rbind(match_home, match_away)
total$date <- as.Date(total$date)

match14 = with(total, total[(date >= "2014-01-01" & date < "2015-01-01"),])
match15 = with(total, total[(date >= "2015-01-01" & date < "2016-01-01"),])
df14 = subset(match14, select=-c(date))
df15 = subset(match15, select=-c(date))

team_ranking14_byscore <- aggregate(df14$score, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(score=x)
team_ranking15_byscore <- aggregate(df15$score, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(score=x)

team_ranking14_bywin <- aggregate(df14$win_rate, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(win_rate=x)
team_ranking15_bywin <- aggregate(df15$win_rate, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(win_rate=x)

team_ranking14 <- merge(team,team_ranking14_byscore, by="team_api_id",all.team_ranking14_byscore = TRUE)
team_ranking15 <- merge(team,team_ranking15_byscore,by="team_api_id",all.team_ranking14_byscore = TRUE)

team_ranking14 <- arrange(merge(team_ranking14,team_ranking14_bywin, by="team_api_id"),-win_rate)
team_ranking15 <- arrange(merge(team_ranking15,team_ranking15_bywin,by="team_api_id"),-win_rate)

head(team_ranking14)
#### John's code
# create dataframes
country <-  data.frame(lDataFrames[1])
league  <-  data.frame(lDataFrames[2])
match   <-  data.frame(lDataFrames[3])
player  <-  data.frame(lDataFrames[4])
team    <-  data.frame(lDataFrames[6])
# select relevant columns
country <- select(country, id, name) %>% 
  rename(country_id = id)  %>% 
  rename(country_name = name)   # use country_id as key for join
league  <- select(league, country_id, name) %>% 
  rename(league_name = name) # use country_id as key for join
match   <- select(match, id, country_id, league_id, season, 
                  stage, date, match_api_id, home_team_api_id, 
                  away_team_api_id, home_team_goal, away_team_goal, 
                  home_player_1, home_player_2, home_player_3, 
                  home_player_4, home_player_5, home_player_6, 
                  home_player_7, home_player_8, home_player_9, 
                  home_player_10, home_player_11, away_player_1, 
                  away_player_2, away_player_3, away_player_4, 
                  away_player_5, away_player_6, away_player_7, 
                  away_player_8, away_player_9, away_player_10, 
                  away_player_11, goal, shoton, shotoff, 
                  foulcommit, card, cross, corner, possession)
player  <- select(player,player_api_id, player_name) # use player_api_id as key for join
team    <- select(team, team_api_id, team_long_name, team_short_name) # use team_api_id as key for join

others = colnames(match)[1:11]
players = colnames(match)[12:33]

flatten <- melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id") %>% 
  mutate(team_api_id = 
           ifelse(grepl("home",variable),home_team_api_id, 
                  ifelse(grepl("away",variable),away_team_api_id,NA))) %>%  
  left_join(country, by = "country_id")    %>% 
  left_join(league,  by = "country_id")    %>%  
  left_join(team,    by = "team_api_id")   %>%
  left_join(player,  by = "player_api_id") %>% 
  separate(season, into = c("season_start","season_end"), sep = "/", convert = TRUE) 
head(flatten)

team_players <- select(flatten, season_start, season_end, 
                       country_name, league_name, player_api_id, 
                       team_api_id, team_long_name, team_short_name,
                       player_name)

team_players_2015 <- team_players %>% filter(season_start == 2015)
team_players_2014 <- team_players %>% filter(season_start == 2014)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




,team_api_id,team_long_name,score,win_rate
,<int>,<chr>,<dbl>,<dbl>
1,9772,SL Benfica,1.633333,0.8333333
2,9823,FC Bayern Munich,2.114286,0.8285714
3,9885,Juventus,1.513514,0.8108108
4,8640,PSV,1.181818,0.7575758
5,8633,Real Madrid CF,2.222222,0.7500000
6,9925,Celtic,1.868421,0.7368421


Warning message in melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id"):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(match). In the next version, this warning will become an error.”


,id,country_id,league_id,season_start,season_end,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,variable,player_api_id,team_api_id,country_name,league_name,team_long_name,team_short_name,player_name
,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,145,1,1,2008,2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,home_player_1,39890,9996,Belgium,Belgium Jupiler League,Royal Excel Mouscron,MOU,Mark Volders
2,146,1,1,2008,2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,home_player_1,38327,8203,Belgium,Belgium Jupiler League,KV Mechelen,MEC,Wouter Biebauw
3,147,1,1,2008,2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,home_player_1,95597,9986,Belgium,Belgium Jupiler League,Sporting Charleroi,CHA,Bertrand Laquait
4,149,1,1,2008,2009,24,2009-03-01 00:00:00,493020,9994,9991,0,1,home_player_1,30934,9994,Belgium,Belgium Jupiler League,Sporting Lokeren,LOK,Boubacar Barry Copa
5,150,1,1,2008,2009,24,2009-02-28 00:00:00,493021,8342,9999,2,1,home_player_1,37990,8342,Belgium,Belgium Jupiler League,Club Brugge KV,CLB,Stijn Stijnen
6,151,1,1,2008,2009,24,2009-02-28 00:00:00,493022,9993,8571,3,0,home_player_1,38391,9993,Belgium,Belgium Jupiler League,Beerschot AC,BAC,Silvio Proto


In [2]:
#### Jaja's code

## Preprocessing Team & Team Attributes
team <- data.frame(lDataFrames[6])
team_attr <- data.frame(lDataFrames[7])

team <- select(team,id,team_api_id)
team_attr$date <- substr(team_attr$date,1,10)
team_attr <- select(team_attr,id,team_api_id,date,
                    buildUpPlaySpeed,buildUpPlayDribbling,
                    buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,
                    chanceCreationShooting,chanceCreationPositioningClass,defencePressure,
                    defenceAggression,defenceTeamWidth,defenceDefenderLineClass)

#filter season date
team_attr_2014 <- subset(team_attr, date >= '2014-07-18' & date <= '2015-05-31')
team_attr_2015 <- subset(team_attr, date >= '2015-07-17' & date <= '2016-05-25')

# remove negative values
team_attr_2014[team_attr_2014 < 0] <- NA
team_attr_2015[team_attr_2015 < 0] <- NA

# merge values of one team to one
aggregate_process <- function(x) {
  if (is.numeric(x)) {mean(x)}
  else {x[1]}
}

team_attr_2014_aggregate <- aggregate(team_attr_2014, 
               by=list(team_attr_2014$team_api_id),
               aggregate_process)
team_attr_2014 <- 
  team_attr_2014_aggregate[2:ncol(team_attr_2014_aggregate)]

team_attr_2015_aggregate <- aggregate(team_attr_2015, 
                                        by=list(team_attr_2015$team_api_id),
                                        aggregate_process)
team_attr_2015 <- 
  team_attr_2015_aggregate[2:ncol(team_attr_2015_aggregate)]

# remove outliers and normalize
preprocess <- function(data) {
  for(i in 1:ncol(data)) {
    if (is.numeric(data[,i]) && (grepl("Width",names(data)[i]) | !grepl("id",names(data)[i]))) {
      mean <- mean(data[,i])
      sd <- sd(data[,i])
      data[,i] <- replace(data[,i],data[,i]>mean+2*sd,NA)
      data[,i] <- replace(data[,i],data[,i]<mean-2*sd,NA)
      data[,i] <- scale(data[,i])
    }
  }
  return(data)
}

team_attr_2014 <- preprocess(team_attr_2014)
team_attr_2015 <- preprocess(team_attr_2015)

# merge with team's rank
team_2014 <- merge(team_ranking14,team_attr_2014[,c(2:14)],
                     by=c('team_api_id'),
                     all.x = TRUE, all.y = TRUE)
team_2015 <- merge(team_ranking15,team_attr_2015[,c(2:14)],
                     by=c('team_api_id'),
                     all.x = TRUE, all.y = TRUE)

team_2014[is.na(team_2014)] <- 0
team_2015[is.na(team_2015)] <- 0

# filter only 3 leagues
team_id_2014 <- team_players_2014$team_api_id
team_2014 <- subset(team_2014,
                      team_api_id %in% team_id_2014)

team_id_2014_eng <- subset(team_players_2014$team_api_id,
  team_players_2014$league_name == "England Premier League")
team_2014_eng <- subset(team_2014,
                      team_api_id %in% team_id_2014_eng)

team_id_2014_ger <- subset(team_players_2014$team_api_id,
  team_players_2014$league_name == "Germany 1. Bundesliga")
team_2014_ger <- subset(team_2014,
                      team_api_id %in% team_id_2014_ger)

team_id_2014_spa <- subset(team_players_2014$team_api_id,
  team_players_2014$league_name == "Spain LIGA BBVA")
team_2014_spa <- subset(team_2014,
                      team_api_id %in% team_id_2014_spa)

team_id_2015 <- team_players_2015$team_api_id
team_2015 <- subset(team_2015,
                      team_api_id %in% team_id_2015)

team_id_2015_eng <- subset(team_players_2015$team_api_id,
  team_players_2015$league_name == "England Premier League")
team_2015_eng <- subset(team_2015,
                      team_api_id %in% team_id_2015_eng)

team_id_2015_ger <- subset(team_players_2015$team_api_id,
  team_players_2015$league_name == "Germany 1. Bundesliga")
team_2015_ger <- subset(team_2015,
                      team_api_id %in% team_id_2015_ger)

team_id_2015_spa <- subset(team_players_2015$team_api_id,
  team_players_2015$league_name == "Spain LIGA BBVA")
team_2015_spa <- subset(team_2015,
                      team_api_id %in% team_id_2015_spa)


# change dummies
team_2014$chanceCreationPositioning_organised <- ifelse(team_2014$chanceCreationPositioningClass == "Organised", 1, 0)
team_2015$chanceCreationPositioning_organised <- ifelse(team_2015$chanceCreationPositioningClass == "Organised", 1, 0)

team_2014$defenceDefenderLine_cover <- ifelse(team_2014$defenceDefenderLineClass == "Cover", 1, 0)
team_2015$defenceDefenderLine_cover <- ifelse(team_2015$defenceDefenderLineClass == "Cover", 1, 0)

# Covarience

team_2014_eng_num <- team_2014_eng[,c(6:11,13:15)]
eng_cor <- cor(team_2014_eng_num, use="complete.obs")
eng_cor

team_2014_ger_num <- team_2014_ger[,c(6:11,13:15)]
ger_cor <- cor(team_2014_ger_num, use="complete.obs")
ger_cor

team_2014_spa_num <- team_2014_spa[,c(6:11,13:15)]
spa_cor <- cor(team_2014_spa_num, use="complete.obs")
spa_cor

# linear regression
library(forecast)


,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
buildUpPlaySpeed,1.00000000,0.58098971,-0.234641751,-0.23703374,-0.42949742,0.092219833,-0.15217649,0.10821657,0.02242014
buildUpPlayDribbling,0.58098971,1.00000000,-0.468380367,-0.38061160,-0.13379112,0.523428343,-0.15720394,-0.13026639,-0.02263399
buildUpPlayPassing,-0.23464175,-0.46838037,1.000000000,0.96571575,0.25891356,-0.008567035,-0.54532918,-0.60695259,-0.42806376
chanceCreationPassing,-0.23703374,-0.38061160,0.965715750,1.00000000,0.24437716,0.029320285,-0.58591298,-0.68314704,-0.57354115
chanceCreationCrossing,-0.42949742,-0.13379112,0.258913561,0.24437716,1.00000000,0.167010747,-0.19010470,-0.08639605,-0.26290236
chanceCreationShooting,0.09221983,0.52342834,-0.008567035,0.02932028,0.16701075,1.000000000,-0.04759354,-0.15501317,0.03388089
defencePressure,-0.15217649,-0.15720394,-0.545329179,-0.58591298,-0.19010470,-0.047593537,1.00000000,0.88756323,0.61050353
defenceAggression,0.10821657,-0.13026639,-0.606952588,-0.68314704,-0.08639605,-0.155013170,0.88756323,1.00000000,0.53923934
defenceTeamWidth,0.02242014,-0.02263399,-0.428063763,-0.57354115,-0.26290236,0.033880895,0.61050353,0.53923934,1.00000000


,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
buildUpPlaySpeed,1.0000000,-0.42488905,0.55163137,0.459410083,0.16287261,0.685489279,-0.15843364,0.233590692,0.689050368
buildUpPlayDribbling,-0.4248891,1.00000000,-0.56596443,0.358527746,-0.07569213,-0.387478551,0.37982359,-0.155385257,-0.521785383
buildUpPlayPassing,0.5516314,-0.56596443,1.00000000,0.168639430,0.03117648,0.535086541,0.19161369,0.511580569,0.208081831
chanceCreationPassing,0.4594101,0.35852775,0.16863943,1.000000000,-0.12633451,0.275607506,0.54312783,0.151016587,0.009133594
chanceCreationCrossing,0.1628726,-0.07569213,0.03117648,-0.126334510,1.00000000,0.221363950,-0.23222596,-0.288950680,0.039298161
chanceCreationShooting,0.6854893,-0.38747855,0.53508654,0.275607506,0.22136395,1.000000000,-0.16061847,0.003802359,0.418761383
defencePressure,-0.1584336,0.37982359,0.19161369,0.543127835,-0.23222596,-0.160618469,1.00000000,0.092952073,-0.299774534
defenceAggression,0.2335907,-0.15538526,0.51158057,0.151016587,-0.28895068,0.003802359,0.09295207,1.000000000,-0.210901726
defenceTeamWidth,0.6890504,-0.52178538,0.20808183,0.009133594,0.03929816,0.418761383,-0.29977453,-0.210901726,1.000000000


,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
buildUpPlaySpeed,1.00000000,0.43794706,0.58098676,0.10674269,0.17737832,0.00712650,-0.31015284,0.17556952,0.02772477
buildUpPlayDribbling,0.43794706,1.00000000,-0.01571592,0.24305950,0.35588634,0.30992597,0.03268473,-0.37722031,0.36791550
buildUpPlayPassing,0.58098676,-0.01571592,1.00000000,-0.05219546,0.07010224,-0.24150471,-0.21376392,0.06971270,-0.25995545
chanceCreationPassing,0.10674269,0.24305950,-0.05219546,1.00000000,0.08104011,0.09534885,0.16565819,-0.07325453,0.21438769
chanceCreationCrossing,0.17737832,0.35588634,0.07010224,0.08104011,1.00000000,-0.35091725,-0.14080797,-0.13813591,0.35425597
chanceCreationShooting,0.00712650,0.30992597,-0.24150471,0.09534885,-0.35091725,1.00000000,0.39168825,0.05788980,0.31407825
defencePressure,-0.31015284,0.03268473,-0.21376392,0.16565819,-0.14080797,0.39168825,1.00000000,-0.22010987,0.23843378
defenceAggression,0.17556952,-0.37722031,0.06971270,-0.07325453,-0.13813591,0.05788980,-0.22010987,1.00000000,0.33842672
defenceTeamWidth,0.02772477,0.36791550,-0.25995545,0.21438769,0.35425597,0.31407825,0.23843378,0.33842672,1.00000000


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [3]:
### England
team_2014_eng_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                   chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+
                    defencePressure+defenceTeamWidth,
                   na.action = na.exclude, data = team_2014_eng)
summary(team_2014_eng_win_rate.lr)

team_2015_eng_win_rate.predict <- predict(team_2014_eng_win_rate.lr, team_2015_eng[,c(6:11,13,15)], level=0.95)
accuracy(team_2015_eng_win_rate.predict, team_2015_eng$win_rate)


Call:
lm(formula = win_rate ~ buildUpPlaySpeed + buildUpPlayDribbling + 
    buildUpPlayPassing + chanceCreationPassing + chanceCreationCrossing + 
    chanceCreationShooting + defencePressure + defenceTeamWidth, 
    data = team_2014_eng, na.action = na.exclude)

Residuals:
       42        70        75        77       109       126       130       132 
-0.033604  0.034846 -0.021001  0.022547  0.088462 -0.024984 -0.013008 -0.070863 
      133       167       219       221       246       247 
 0.024657  0.080249  0.007622 -0.091637 -0.038801  0.035513 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)   
(Intercept)             0.695639   0.126211   5.512  0.00269 **
buildUpPlaySpeed       -0.113790   0.038989  -2.919  0.03307 * 
buildUpPlayDribbling    0.252808   0.063312   3.993  0.01039 * 
buildUpPlayPassing      0.154114   0.113873   1.353  0.23390   
chanceCreationPassing  -0.207125   0.122542  -1.690  0.15178   
chanceCreationCrossing  0.008679   0.034

,ME,RMSE,MAE,MPE,MAPE
Test set,-0.2490629,0.3908586,0.3180827,-88.61254,106.3014


In [4]:
### Germany
team_2014_ger_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                   chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+
                   defencePressure+defenceAggression,
                   na.action = na.exclude, data = team_2014_ger)
summary(team_2014_ger_win_rate.lr)

team_2015_ger_win_rate.predict <- predict(team_2014_ger_win_rate.lr, team_2015_ger[,c(6:15)], level=0.95)
accuracy(team_2015_ger_win_rate.predict, team_2015_ger$win_rate)


Call:
lm(formula = win_rate ~ buildUpPlaySpeed + buildUpPlayDribbling + 
    buildUpPlayPassing + chanceCreationPassing + chanceCreationCrossing + 
    chanceCreationShooting + defencePressure + defenceAggression, 
    data = team_2014_ger, na.action = na.exclude)

Residuals:
      38       44       60       67       72      140      141      142 
-0.05640  0.10091 -0.06066  0.01745  0.05064  0.06194  0.10894 -0.11439 
     162      194      195      223      252 
-0.07397  0.01554 -0.00183  0.05658 -0.10474 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)   
(Intercept)             0.40629    0.08365   4.857   0.0083 **
buildUpPlaySpeed        0.03457    0.06842   0.505   0.6400   
buildUpPlayDribbling   -0.06090    0.07855  -0.775   0.4815   
buildUpPlayPassing     -0.04071    0.09623  -0.423   0.6940   
chanceCreationPassing  -0.02110    0.07440  -0.284   0.7908   
chanceCreationCrossing  0.06325    0.04246   1.490   0.2106   
chanceCreationShooting -0.06

,ME,RMSE,MAE,MPE,MAPE
Test set,-0.1256309,0.2176063,0.1785233,-47.82198,65.60107


In [5]:
### Spain
team_2014_spa_win_rate.lr <- lm(win_rate ~ buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing+
                   chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+
                   defenceAggression+defenceTeamWidth,
                   na.action = na.exclude, data = team_2014_spa)
summary(team_2014_spa_win_rate.lr)

team_2015_spa_win_rate.predict <- predict(team_2014_spa_win_rate.lr, team_2015_spa[,c(6:15)], level=0.95)
accuracy(team_2015_spa_win_rate.predict, team_2015_spa$win_rate)


Call:
lm(formula = win_rate ~ buildUpPlaySpeed + buildUpPlayDribbling + 
    buildUpPlayPassing + chanceCreationPassing + chanceCreationCrossing + 
    chanceCreationShooting + defenceAggression + defenceTeamWidth, 
    data = team_2014_spa, na.action = na.exclude)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.21603 -0.06565 -0.01679  0.10686  0.18222 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             0.406959   0.078599   5.178 0.000581 ***
buildUpPlaySpeed        0.129462   0.085308   1.518 0.163432    
buildUpPlayDribbling   -0.196758   0.116040  -1.696 0.124198    
buildUpPlayPassing     -0.054806   0.050568  -1.084 0.306633    
chanceCreationPassing   0.071980   0.043711   1.647 0.134019    
chanceCreationCrossing -0.009737   0.055808  -0.174 0.865358    
chanceCreationShooting  0.006980   0.045914   0.152 0.882522    
defenceAggression      -0.104085   0.076784  -1.356 0.208275    
defenceTeamWidth        0.096

,ME,RMSE,MAE,MPE,MAPE
Test set,-0.005711542,0.2884549,0.2162438,-27.22879,71.0349
